# Lab #1 Images, optics, and the statistics of light
## Section 3.2

In [ ]:
# Import necessary packages
from glob import glob
import numpy as np
import os
import astropy.io.fits as fits
import matplotlib.pyplot as plt
from astropy.visualization import PercentileInterval, ImageNormalize, SqrtStretch
from scipy.optimize import curve_fit
from photutils.aperture import CircularAperture, CircularAnnulus, ApertureStats, aperture_photometry
import matplotlib.ticker as ticker

In [ ]:
# Important analytical values
#gain =
#bias = 
#dark_current = 
#read_noise = 
#plate_scale =


# Astronomical Measurements
## 2.3.2 Astrometry
**Astrometry:** the study of the precise measurement of the positions and movements of celestial objects.

**KEY STEPS:**
1) Measure the position and positional uncertainty of a star in the globular cluster-sim.
2) Determine the astrometric precision empirically and estimate analytically.


In [ ]:
# star_cluster_directory =# Path to star cluster files
star_cluster_files = glob(os.path.join(star_cluster_directory, '*.fits'))

print(f"\nFound {len(star_cluster_files)} files in '{star_cluster_directory}'.")
print(f"Directory exposure time is {(os.path.basename(star_cluster_files[0]).split('_')[1].split('.')[0])}.\n")

In [ ]:
sum_star_cluster_data = np.zeros_like(fits.getdata(star_cluster_files[0]))

for star_cluster_file in star_cluster_files:
    star_cluster_data = fits.getdata(star_cluster_file) * gain
    
    sum_star_cluster_data += star_cluster_data

mean_star_cluster_data = sum_star_cluster_data / len(star_cluster_files)

interval = PercentileInterval(99) # Get rid of extreme outliers
vmin, vmax = interval.get_limits(mean_star_cluster_data)
norm = ImageNormalize(mean_star_cluster_data, vmin=vmin, vmax=vmax, stretch=SqrtStretch())

plt.figure(figsize=(6, 6))
plt.title('Unprocessed Image of Star Cluster')
plt.xlabel('X Index (pixels)')
plt.ylabel('Y Index (pixels)')
plt.imshow(mean_star_cluster_data, cmap='gist_heat', origin='lower')
plt.show()

In [ ]:
#ylow = 
#yhigh = 
#xlow = 
#xhigh =

mean_star_data = mean_star_cluster_data[ylow:yhigh, xlow:xhigh] # zoom-in on star of interest

plt.figure(figsize=(4, 4))
plt.title('Normalized Mean Image of Star')
plt.xlabel('X Index (pixels)')
plt.ylabel('Y Index (pixels)')
plt.imshow(mean_star_data, norm=norm, cmap='gist_heat', origin='lower')
plt.show()

### Gaussian Function
A star is a point source, however, a detector can only get as sharp as its **diffraction-limit**.

$$
\theta = 1.22 \frac{\lambda}{D_{\text{telescope}}} \, \text{radians}
$$

Therefore, the point source while turn into a **point-spread function**, modeled by a Gaussian distribution:

$$
f(x, y) = A * \text{exp}\left({-\frac{(x - x_0)^2}{2\sigma_x^2}} - \frac{(y - y_0)^2}{2\sigma_y^2}\right)
$$

Where $A$ is the amplitude (or peak flux), ($x_0$, $y_0$) is the center of the function (star), $\sigma_x$ and $\sigma_y$ are the standard deviations. To find the FWHM we need to know the following relationship:

$$
\text{FWHM} = 2\sqrt{2\ln(2)}\sigma = 2.355\sigma
$$

In [ ]:
# 2D Gaussian function for determining FWHM of star
def gaussian_2d(xy, amplitude, xo, yo, sigma_x, sigma_y, theta):
    x, y = xy
    xo = float(xo)
    yo = float(yo)    
    a = (np.cos(theta)**2) / (2*sigma_x**2) + (np.sin(theta)**2)/(2*sigma_y**2)
    b = -(np.sin(2*theta)) / (4*sigma_x**2) + (np.sin(2*theta))/(4*sigma_y**2)
    c = (np.sin(theta)**2) / (2*sigma_x**2) + (np.cos(theta)**2)/(2*sigma_y**2)
    g = amplitude*np.exp( - (a*((x-xo)**2) + 2*b*(x-xo)*(y-yo) 
                            + c*((y-yo)**2)))
    return g.ravel()

In [ ]:
#guess_center_star =  # (y, x)

# Create range of values for the Gaussian fit
x = np.arange(0, mean_star_data.shape[1])
y = np.arange(0, mean_star_data.shape[0])
x, y =np.meshgrid(x, y)

initial_guess_for_gaussian_fit = (np.max(mean_star_data), guess_center_star[1], guess_center_star[0], 5, 5, 0)

popt, pcov = curve_fit(gaussian_2d, (x, y), mean_star_data.ravel(), p0=initial_guess_for_gaussian_fit)
amplitude, x0, y0, sigma_x, sigma_y, offset = popt

center = np.array([x0 + 5, y0 - 10])

fwhm_x = 2.355 * sigma_x
fwhm_y = 2.355 * sigma_y

print(f"\nCentroid of star is at pixel ({np.round(center[0])}, {np.round(center[1])}).")
print(f"FWHM in x is {np.round(fwhm_x)} pixels and in y is {np.round(fwhm_y)} pixels.\n")

In [ ]:
star_fitted_data = gaussian_2d((x, y), *popt)

fig, ax = plt.subplots(figsize=(4, 4))
ax.set_title('Mean Image of Star w/ Contours')
ax.set_xlabel('X Index (pixels)')
ax.set_ylabel('Y Index (pixels)')
ax.imshow(mean_star_data, cmap='gist_heat', origin='lower')
ax.contour(x, y, mean_star_data, 5, colors='w')
ax.scatter(center[0], center[1], color='green')
plt.show()

## 2.3.2.1 Empirical Astrometric Precision

In [ ]:
centroid_signal = mean_star_data[int(y0), int(x0)]
centroid_noise = np.sqrt(centroid_signal)
empirical_signal_to_noise_ratio = centroid_signal / centroid_noise
empirical_astrometric_uncertainty = ((fwhm_x + fwhm_y)/2) / empirical_signal_to_noise_ratio

print(f"\nThe empirical signal-to-noise ratio is {np.round(empirical_signal_to_noise_ratio, 2)}.")
print(f"The star's position is ({np.round(center[0])}, {np.round(center[1])}) +/- {np.round(empirical_astrometric_uncertainty)} pixels.\n")
print(f"The star's position is ({np.round(center[0] * plate_scale)}, {np.round(center[1] * plate_scale)}) +/- {np.round(analytical_astrometric_uncertainty * plate_scale)} arcseconds.\n")


## 2.3.2.2 Analytical Astrometric Precision

In [ ]:
star_aperture = CircularAperture(center, r=35)
annulus_aperture = CircularAnnulus(center, r_in=70, r_out=100)

star_aperture.plot(color='white', lw=2)
annulus_aperture.plot(color='red', lw=2)
plt.imshow(mean_star_data, cmap='gist_heat', origin='lower')
plt.show()

In [ ]:
aperture_stats = ApertureStats(mean_star_data, star_aperture)
annulus_stats = ApertureStats(mean_star_data, annulus_aperture)
aperture_area = star_aperture.area_overlap(mean_star_data)

sum_signal = aperture_stats.sum
mean_background = annulus_stats.mean

print(f"\nMeasured signal from the star is {np.round(sum_signal)} (electrons).")
print(f"Background is {np.round(mean_background)} (electrons/pixel).\n")

In [ ]:
measured_signal = sum_signal - (mean_background * aperture_area) - (bias) - (dark_current * aperture_area)
uncertainty_measured_signal = np.sqrt(sum_signal - (mean_background * aperture_area) - bias - (dark_current * aperture_area))

analytical_signal_to_noise_ratio = measured_signal / uncertainty_measured_signal
analytical_astrometric_uncertainty = ((fwhm_x + fwhm_y) / 2) / analytical_signal_to_noise_ratio

print(f"\nThe empirical signal-to-noise ratio is {np.round(analytical_signal_to_noise_ratio, 2)}.")
print(f"The star's position is ({np.round(center[0])}, {np.round(center[1])}) +/- {np.round(analytical_astrometric_uncertainty)} pixels.\n")
print(f"The star's position is ({np.round(center[0] * plate_scale)}, {np.round(center[1] * plate_scale)}) +/- {np.round(analytical_astrometric_uncertainty * plate_scale)} arcseconds.\n")

